# RNN
This notebook contains scraches of RNNs

# Load Data

In [1]:
import utils

  _________________________________
| Feed me some grass! Ik heb honger |
                                 \
                                  \
                                    ^__^
                                    (oo)\_______
                                    (__)\       )\/\
                                        ||----w |
                                        ||     ||


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
dirpath_left_nopunch = "Data/Raw Data/Preliminary/Preliminary Left No Punch/"
dirpath_right_nopunch = "Data/Raw Data/Preliminary/Preliminary Right No Punch/"
dirpath_jab = "Data/Raw Data/Preliminary/Preliminary Jab/"
dirpath_cross = "Data/Raw Data/Preliminary/Preliminary Cross/"
dirpath_lh = "Data/Raw Data/Preliminary/Preliminary Lead Hook"
dirpath_rh = "Data/Raw Data/Preliminary/Preliminary Rear Hook/"
dirpath_lu = "Data/Raw Data/Preliminary/Preliminary Lead Uppercut/"
dirpath_ru = "Data/Raw Data/Preliminary/Preliminary Rear Uppercut/"

dirpath_savefig = "Figures/"

In [5]:
dirpath_left_nopunch = "Data/Raw Data/Preliminary/Preliminary Left No Punch/"
dirpath_right_nopunch = "Data/Raw Data/Preliminary/Preliminary Right No Punch/"
dirpath_jab = "Data/Raw Data/Preliminary/Preliminary Jab/"
dirpath_cross = "Data/Raw Data/Preliminary/Preliminary Cross/"
dirpath_lh = "Data/Raw Data/Preliminary/Preliminary Lead Hook"
dirpath_rh = "Data/Raw Data/Preliminary/Preliminary Rear Hook/"
dirpath_lu = "Data/Raw Data/Preliminary/Preliminary Lead Uppercut/"
dirpath_ru = "Data/Raw Data/Preliminary/Preliminary Rear Uppercut/"

dirpath_savefig = "Figures/"

In [6]:
# Left hand data without punch
pre_lft_nopunch_acc = pd.read_csv(os.path.join(dirpath_left_nopunch,"Accelerometer.csv"))
pre_lft_nopunch_gyr = pd.read_csv(os.path.join(dirpath_left_nopunch,"Gyroscope.csv"))
pre_lft_nopunch_bar = pd.read_csv(os.path.join(dirpath_left_nopunch,"Barometer.csv"))

# Right hand data without punch
pre_rght_nopunch_acc = pd.read_csv(os.path.join(dirpath_right_nopunch,"Accelerometer.csv"))
pre_rght_nopunch_gyr = pd.read_csv(os.path.join(dirpath_right_nopunch,"Gyroscope.csv"))
pre_rght_nopunch_bar = pd.read_csv(os.path.join(dirpath_right_nopunch,"Barometer.csv"))

# Jab data
pre_jab_acc = pd.read_csv(os.path.join(dirpath_jab,"Accelerometer.csv"))
pre_jab_gyr = pd.read_csv(os.path.join(dirpath_jab,"Gyroscope.csv"))
pre_jab_bar = pd.read_csv(os.path.join(dirpath_jab,"Barometer.csv"))

# Cross data
pre_cross_acc = pd.read_csv(os.path.join(dirpath_cross,"Accelerometer.csv"))
pre_cross_gyr = pd.read_csv(os.path.join(dirpath_cross,"Gyroscope.csv"))
pre_cross_bar = pd.read_csv(os.path.join(dirpath_cross,"Barometer.csv"))

# Lead Hook data
pre_lh_acc = pd.read_csv(os.path.join(dirpath_lh,"Accelerometer.csv"))
pre_lh_gyr = pd.read_csv(os.path.join(dirpath_lh,"Gyroscope.csv"))
pre_lh_bar = pd.read_csv(os.path.join(dirpath_lh,"Barometer.csv"))

# Rear Hook data
pre_rh_acc = pd.read_csv(os.path.join(dirpath_rh,"Accelerometer.csv"))
pre_rh_gyr = pd.read_csv(os.path.join(dirpath_rh,"Gyroscope.csv"))
pre_rh_bar = pd.read_csv(os.path.join(dirpath_rh,"Barometer.csv"))

# Lead Uppercut data
pre_lu_acc = pd.read_csv(os.path.join(dirpath_lu,"Accelerometer.csv"))
pre_lu_gyr = pd.read_csv(os.path.join(dirpath_lu,"Gyroscope.csv"))
pre_lu_bar = pd.read_csv(os.path.join(dirpath_lu,"Barometer.csv"))

# Rear Uppercut data
pre_ru_acc = pd.read_csv(os.path.join(dirpath_ru,"Accelerometer.csv"))
pre_ru_gyr = pd.read_csv(os.path.join(dirpath_ru,"Gyroscope.csv"))
pre_ru_bar = pd.read_csv(os.path.join(dirpath_ru,"Barometer.csv"))

In [7]:
# Use dictionary to store all dataframe to simplify the preprocessing later
dict_pre_data = {
    "pre_lft_nopunch_acc" : pre_lft_nopunch_acc,
    "pre_lft_nopunch_gyr" : pre_lft_nopunch_gyr,
    "pre_lft_nopunch_bar" : pre_lft_nopunch_bar,
    "pre_rght_nopunch_acc" : pre_rght_nopunch_acc,
    "pre_rght_nopunch_gyr" : pre_rght_nopunch_gyr,
    "pre_rght_nopunch_bar" : pre_rght_nopunch_bar,    
    "pre_jab_acc" : pre_jab_acc,
    "pre_jab_gyr" : pre_jab_gyr,
    "pre_jab_bar" : pre_jab_bar,    
    "pre_cross_acc" : pre_cross_acc,
    "pre_cross_gyr" : pre_cross_gyr,
    "pre_cross_bar" : pre_cross_bar,
    "pre_lh_acc" : pre_lh_acc,
    "pre_lh_gyr" : pre_lh_gyr,
    "pre_lh_bar" : pre_lh_bar,
    "pre_rh_acc" : pre_rh_acc,
    "pre_rh_gyr" : pre_rh_gyr,
    "pre_rh_bar" : pre_rh_bar,
    "pre_lu_acc" : pre_lu_acc,
    "pre_lu_gyr" : pre_lu_gyr,
    "pre_lu_bar" : pre_lu_bar,
    "pre_ru_acc" : pre_ru_acc,
    "pre_ru_gyr" : pre_ru_gyr,
    "pre_ru_bar" : pre_ru_bar,}

In [8]:
# Import the transformed dataset

##### Change the filepaths manually according to the path on your own device if necessary.

dict_transformed_df = dict()

for transformed_df in os.listdir("Data/Preprocessed Data/"):
    df_name = transformed_df.replace(".csv", "").replace("transformed_","")
    df_content = pd.read_csv(os.path.join("Data/Preprocessed Data/",transformed_df))
    dict_transformed_df.setdefault(df_name, df_content)

# Vanilla RNN
The standard RNN. 

# LSTM

An optimized version of RNN that evaluates the importance of information passed to the Network by Tanh and Sigmoid function and selectively filters out the information that are considered less important. This method is famous for its ability in avoiding vanishing/exploring Gradient Problem.

LSTM pipeline:
- Data Preprocessing: Standardization and Convering data into sequences of 5 datapoints. LSTM will study the pattern in these 5 sequences.
- - e.g.: We give a sequence of 5 jabs to LSTM, LSTM will know that *On this is how Jabs look like!* And then learn to make this prediction
- Train-test split
- One-hot encode the label
- Train the model
- Evaluation

### Data Preprocessing

In [9]:
def drop_tails(df, seq_length):
    
    df_length = len(df) // seq_length * seq_length
    return df.iloc[:df_length]

# Convert data into sequences
def create_sequential_data(df, sequence_length):
    x = []
    y = []
    features = df.iloc[:,:-1]
    labels = df.iloc[:,-1]

    num_sequences = len(df) // sequence_length

    sequence_start_idx = 0 
    sequence_end_idx = sequence_length

    while num_sequences > 0:
        x.append(features[sequence_start_idx:sequence_end_idx])
        y.append(labels.iloc[sequence_start_idx])

        # print(num_sequences, sequence_start_idx, sequence_end_idx)
        num_sequences -= 1
        sequence_start_idx += sequence_length
        sequence_end_idx += sequence_length 

    return np.array(x), np.array(y)

In [10]:
df_whole = pd.concat([drop_tails(df, 5) for df in dict_transformed_df.values()], axis=0)
df_whole = df_whole.drop(columns='X (hPa)')
df_whole

,X (m/s^2),Y (m/s^2),Z (m/s^2),X (rad/s),Y (rad/s),Z (rad/s),Punch Type
0,6.137,6.095,3.791,0.206,0.902,0.285,cross
1,14.568,28.642,10.195,2.934,5.756,2.912,cross
2,12.615,22.545,10.761,2.541,7.395,2.865,cross
3,13.644,21.431,8.318,2.365,6.340,2.121,cross
4,13.778,29.763,11.123,3.318,8.487,3.265,cross
...,...,...,...,...,...,...,...
295,7.231,17.051,26.618,4.463,3.269,1.997,ru
296,6.853,16.481,22.919,4.591,3.069,2.327,ru
297,6.688,14.249,20.703,4.112,4.443,2.806,ru
298,5.891,11.743,19.362,2.780,4.224,2.727,ru


In [11]:
seq_data, seq_label = create_sequential_data(df_whole, 5)

In [12]:
# Encode the label
label_encoder = OneHotEncoder(sparse_output=False)

seq_label = label_encoder.fit_transform(seq_label.reshape(-1,1))

In [13]:
# Perform train test split and normalize the training dataset
x_train, x_test, y_train, y_test = train_test_split(seq_data, seq_label, test_size=0.25, random_state=50)

# scaler = MinMaxScaler()
# x_train = scaler.fit_transform(x_train.reshape(-1,1))
# x_test = scaler.fit_transform(x_test.reshape(-1,1))

y_train.shape

(306, 8)

### Construct the model

In [17]:
lstm_model = Sequential()

lstm_model.add(LSTM(units=8, input_shape=(x_train.shape[1], x_train.shape[2])))

lstm_model.add(Dropout(0.5))

lstm_model.add(Dense(y_train.shape[1], activation='softmax'))

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 8)                 480       
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
Total params: 552 (2.16 KB)
Trainable params: 552 (2.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
history = lstm_model.fit(x_train, y_train, epochs=40, batch_size=5, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/40


49/49 [==============================] - 2s 10ms/step - loss: 2.1175 - accuracy: 0.1680 - val_loss: 2.0502 - val_accuracy: 0.2742
Epoch 2/40
49/49 [==============================] - 0s 3ms/step - loss: 2.0416 - accuracy: 0.2213 - val_loss: 1.9753 - val_accuracy: 0.3065
Epoch 3/40
49/49 [==============================] - 0s 3ms/step - loss: 1.9869 - accuracy: 0.2500 - val_loss: 1.9151 - val_accuracy: 0.3226
Epoch 4/40
49/49 [==============================] - 0s 3ms/step - loss: 1.8650 - accuracy: 0.2705 - val_loss: 1.8382 - val_accuracy: 0.4032
Epoch 5/40
49/49 [==============================] - 0s 3ms/step - loss: 1.8012 - accuracy: 0.3320 - val_loss: 1.7512 - val_accuracy: 0.3065
Epoch 6/40
49/49 [==============================] - 0s 3ms/step - loss: 1.6827 - accuracy: 0.3525 - val_loss: 1.6296 - val_accuracy: 0.3065
Epoch 7/40
49/49 [==============================] - 0s 3ms/step - loss: 1.6075 - accuracy: 0.3484 - val_loss: 1.5456 - val_accuracy: 0.3387
Epoch 8/40
49/49 

In [20]:
# Assuming x_test and y_test are your unseen test data
test_loss, test_accuracy = lstm_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

4/4 [==============================] - 0s 2ms/step - loss: 0.7630 - accuracy: 0.7255
Test Accuracy: 0.7255


In [21]:
from sklearn.metrics import confusion_matrix, classification_report

# Predictions on test data
y_pred = lstm_model.predict(x_test)
# Convert predicted probabilities to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
# Convert one-hot encoded true labels to class labels
y_true_classes = np.argmax(y_test, axis=1)

# Confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:")
print(cm)

# Classification report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes))


4/4 [==============================] - 0s 2ms/step
Confusion Matrix:
[[15  3  0  1  0  0  0  0]
 [ 4 15  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  5  0]
 [ 0  0  0 15  0  0  2  0]
 [ 0  0  0  0  9  0  0  5]
 [ 0  0  0  3  0  1  0  0]
 [ 0  0  0  0  0  0 11  0]
 [ 0  0  0  0  3  0  1  8]]

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        19
           1       0.83      0.79      0.81        19
           2       0.00      0.00      0.00         6
           3       0.75      0.88      0.81        17
           4       0.75      0.64      0.69        14
           5       1.00      0.25      0.40         4
           6       0.58      1.00      0.73        11
           7       0.62      0.67      0.64        12

    accuracy                           0.73       102
   macro avg       0.66      0.63      0.61       102
weighted avg       0.70      0.73      0.70       102



C:\Users\yixin\Desktop\Workspace\python venv\MLQS-venu\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\yixin\Desktop\Workspace\python venv\MLQS-venu\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\yixin\Desktop\Workspace\python venv\MLQS-venu\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo